In [1]:
import math
import re
from   random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
my_list   = ['apple', 'banana', 'strawberry']
my_list_2 = 'kaunghtetcho'

for i,x in enumerate(my_list):
    print('i: ', i)
    print('x: ', x)

for i,x in enumerate(my_list_2):
    print('i: ', i)
    print('x: ', x)

    

i:  0
x:  apple
i:  1
x:  banana
i:  2
x:  strawberry
i:  0
x:  k
i:  1
x:  a
i:  2
x:  u
i:  3
x:  n
i:  4
x:  g
i:  5
x:  h
i:  6
x:  t
i:  7
x:  e
i:  8
x:  t
i:  9
x:  c
i:  10
x:  h
i:  11
x:  o


In [ ]:
l1 = ["eat", "sleep", "repeat"]
s1 = "geek"
 
# creating enumerate objects
obj1 = enumerate(l1)
obj2 = enumerate(s1)
 
print ("Return type:", type(obj1))
print (list(enumerate(l1)))
 
# changing start index to 2 from 0
print (list(enumerate(s1, 2)))

In [ ]:
import spacy

# Load the English language model
nlp = spacy.load('en_core_web_sm')

# Define some raw text
raw_text = "I love natural language processing. It's fascinating!"

# Process the raw text
doc = nlp(raw_text)

# Iterate over each sentence in the processed document and print it
for sent in doc.sents:
    print(sent)


In [ ]:
import spacy

# Load the English language model
nlp = spacy.load('en_core_web_sm')

# Define some raw text
raw_text = "Apple is looking at buying U.K. startup for $1 billion"

# Process the raw text
doc = nlp(raw_text)

# Collect all named entities into a list
sentences = list(doc.ents)

print(sentences)

In [ ]:
# Dummy values for token_a and token_b
token_a = ["apple", "banana", "orange"]  # Contains 3 tokens
token_b = ["cat", "dog"]  # Contains 2 tokens

# Constructing segment_id
segment_id = [0] * (1 + len(token_a) + 1) + [1] * (len(token_b) + 1)

# Printing segment_id
print(segment_id)

In [3]:
input_ids = [101, 2054, 2003, 103, 2012, 102, 2054, 2003, 102]
word2id = {'[CLS]': 101, '[SEP]': 102, 'apple': 2054, 'banana': 2003}

# (0, 101), (1, 2054), (2, 2003), (3, 103), (4, 2012), (5, 102), (6, 2054), (7, 2003), (8, 102)

candidates_masked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] and token != word2id['[SEP]']]

candidates_masked_pos # 1,2,3,4,6,7 are not special tokens

[1, 2, 3, 4, 6, 7]

In [5]:
max_mask =  15

In [6]:
# Calculate 15% of the total number of tokens
percentage = len(input_ids) * 0.15

# Round the percentage to the nearest integer
rounded_percentage = round(percentage)

# Ensure the rounded percentage is at least 1
if rounded_percentage < 1:
    n_pred = 1
else:
    n_pred = rounded_percentage

# Ensure n_pred does not exceed max_mask
if n_pred > max_mask:
    n_pred = max_mask

In [7]:
shuffle(candidates_masked_pos)  # Shuffling the list in place

print(candidates_masked_pos)  # Printing the shuffled list

[7, 4, 3, 6, 1, 2]


In [8]:
for i,token in enumerate(input_ids):
    if token!= word2id['[CLS]'] and token != word2id['[SEP]']:
        print(i)

1
2
3
4
6
7


In [9]:
for i,token in enumerate(input_ids):
    print(i)
    print(token)

0
101
1
2054
2
2003
3
103
4
2012
5
102
6
2054
7
2003
8
102


In [10]:
candidates_masked_pos

[7, 4, 3, 6, 1, 2]

In [11]:
n_pred

1

In [12]:
masked_tokens, masked_pos = [], []

for pos in candidates_masked_pos[:n_pred]:

    masked_pos.append(pos)
    masked_tokens.append(input_ids[pos])

    print(masked_pos)
    print(masked_tokens)

[7]
[2003]


In [13]:
len(input_ids)

9

In [14]:
max_len = 1000
n_pad = max_len - len(input_ids)
# input_ids.extend([0] * n_pad)
# segment_id.extend([0] * n_pad)

print(n_pad)

991


In [ ]:
len(input_ids)

In [16]:
# Example input_ids list
input_ids = [101, 2054, 2003, 103]

# Example value of n_pad (number of zeros to add)
n_pad = 3

# Extending input_ids with n_pad zeros
input_ids.extend([0]*n_pad)

# Printing the updated input_ids list
print(input_ids)


[101, 2054, 2003, 103, 0, 0, 0]


In [17]:
# Example data
max_mask = 10
n_pred   = 6
masked_tokens = [101, 2054, 2003, 103]
masked_pos    = [0, 1, 2, 3]

# Check if padding is needed
if max_mask > n_pred:
    n_pad = max_mask - n_pred
    masked_tokens.extend([0] * n_pad)
    masked_pos.extend([0] * n_pad)

# Print the updated masked_tokens and masked_pos
print("Updated masked_tokens:", masked_tokens)
print("Updated masked_pos:", masked_pos)


Updated masked_tokens: [101, 2054, 2003, 103, 0, 0, 0, 0]
Updated masked_pos: [0, 1, 2, 3, 0, 0, 0, 0]


In [18]:
input_ids.extend([0] * n_pad)

In [19]:
len(segment_id)

NameError: name 'segment_id' is not defined

In [20]:
input_ids  = [101, 2054, 2003, 103, 2012, 102, 2054, 2003, 102]
word2id    = {'[CLS]': 101, '[SEP]': 102, 'apple': 2054, 'banana': 2003, '[MASK]': 103}
id2word    = {2054: 'apple', 2003: 'banana'}
vocab_size = 5000
n_pred = 3  # Example number of predictions

from random import shuffle, random, randint

# Code for generating masked tokens
cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] and token != word2id['[SEP]']]
shuffle(cand_maked_pos)
masked_tokens, masked_pos = [], []

for pos in cand_maked_pos[:n_pred]:
    masked_pos.append(pos)  # Remember the position
    masked_tokens.append(input_ids[pos])  # Remember the tokens
    
    rand_num = random()
    if rand_num < 0.1:  # 10%
        index = randint(0, vocab_size - 1)  # Random index in vocabulary
        input_ids[pos] = word2id[id2word[index]]  # Replace with random token
    elif rand_num < 0.9:  # 80%
        input_ids[pos] = word2id['[MASK]']  # Make mask
    else:  # 10% (do nothing)
        pass

print("Original Input IDs:", input_ids)
print("Masked Positions:", masked_pos)
print("Masked Tokens:", masked_tokens)


Original Input IDs: [101, 103, 2003, 103, 103, 102, 2054, 2003, 102]
Masked Positions: [3, 4, 1]
Masked Tokens: [103, 2012, 2054]


In [21]:
languages = ['Java', 'Python', 'JavaScript']
versions = [14, 3, 6]

result = zip(languages, versions)
print(list(result))
print(type(result))

[('Java', 14), ('Python', 3), ('JavaScript', 6)]
<class 'zip'>


In [22]:
coordinate = ['x', 'y', 'z']
value = [3, 4, 5]

result = zip(coordinate, value)
result_list = list(result)
print(result_list)

c, v =  zip(*result_list)
print('c: ', c), print(type(c))
print('v: ', v)

[('x', 3), ('y', 4), ('z', 5)]
c:  ('x', 'y', 'z')
<class 'tuple'>
v:  (3, 4, 5)


In [ ]:
import torch
import torch.nn as nn

# Define the Embedding class
class Embedding(nn.Module):
    def __init__(self, vocab_size, max_len, n_segments, d_model):
        super(Embedding, self).__init__()
        self.tok_embd = nn.Embedding(vocab_size, d_model)
        self.pos_embd = nn.Embedding(max_len, d_model)
        self.seg_embd = nn.Embedding(n_segments, d_model)
        self.norm     = nn.LayerNorm(d_model)
        
    def forward(self, x, seg):
        # x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        embedding = self.tok_embd(x) + self.pos_embd(pos) + self.seg_embd(seg)
        return self.norm(embedding)

# Example data
vocab_size = 10000  # Vocabulary size
max_len = 20        # Maximum sequence length
n_segments = 2      # Number of segments (e.g., sentences)
d_model = 768       # Embedding size

# Instantiate the Embedding model
embedding_model = Embedding(vocab_size, max_len, n_segments, d_model)

# Example input data
input_ids = torch.tensor([[101, 2054, 2003, 102, 0], [201, 202, 102, 0, 0]])  # Batch of input token IDs
segment_ids = torch.tensor([[0, 0, 0, 0, 0], [0, 0, 1, 1, 1]])  # Batch of segment IDs

# Forward pass through the Embedding model
embeddings = embedding_model(input_ids, segment_ids)

# Print the shape of the output embeddings
print("Output embeddings shape:", embeddings.shape)


In [ ]:
import torch

# Example sequence lengths
batch_size = 2
len_q = 4  # Length of query sequence
len_k = 5  # Length of key sequence

# Example query sequence (with padding)
seq_q = torch.tensor([
    [101, 2054, 2003, 0],  # Padded with 0
    [201, 202, 102, 0]     # Padded with 0
])

# Example key sequence (with padding)
seq_k = torch.tensor([
    [101, 2054, 2003, 102, 0],  # Padded with 0
    [201, 202, 102, 0, 0]        # Padded with 0
])

# Call the get_attn_pad_mask function
pad_attn_mask = get_attn_pad_mask(seq_q, seq_k)

# Print the resulting attention mask
print("Attention Mask:")
print(pad_attn_mask)


In [ ]:
import torch
import torch.nn as nn

# Define the Embedding class
class Embedding(nn.Module):
    def __init__(self, vocab_size, max_len, n_segments, d_model):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_len, d_model)
        self.seg_embed = nn.Embedding(n_segments, d_model)
        self.norm = nn.LayerNorm(d_model)
        
    def forward(self, x, seg):
        # x, seg: (bs, len)
        seq_len = x.size(1)
        print('seq_len: ', seq_len)
        pos = torch.arange(seq_len, dtype=torch.long)  # Create tensor containing indices from 0 to seq_len-1
        print('pos: ', pos)
        pos = pos.unsqueeze(0).expand_as(x)  # Expand pos to match the shape of x
        print('pos after unsqueeze: ', pos)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        print('embedding: ', embedding.shape)
        return self.norm(embedding)

# Example data
vocab_size = 10000  # Vocabulary size
max_len = 10        # Maximum sequence length
n_segments = 2      # Number of segments
d_model = 768       # Embedding size

# Instantiate the Embedding model
embedding_model = Embedding(vocab_size, max_len, n_segments, d_model)

# Example input data
# Batch size is 2, sequence length is 5
x = torch.tensor([[1, 2, 3, 0, 0], [4, 5, 6, 7, 0]])  # Input token IDs
seg = torch.tensor([[0, 0, 1, 1, 1], [0, 0, 0, 1, 1]])  # Segment IDs

# Forward pass through the Embedding model
output = embedding_model(x, seg)

# Print the output shape
print("Output shape:", output.shape)

print('x.size(0): ',x.size(0))
print('x.size(1): ',x.size(1))
# size() method returns a tuple representing the size of each dimension of the tensor.
# batch_size, dimension

In [6]:
# numpy.size() method 
 
# importing numpy 
import numpy as np
 
# Making a random array 
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
 
# By default, give the total number of elements.
print(np.size(arr, 0))
print(np.size(arr, 1))

2
4


In [7]:
seq_len = x.size(1)
seq_len

5

In [8]:
pos = torch.arange(seq_len, dtype=torch.long)
pos

tensor([0, 1, 2, 3, 4])

In [ ]:
pos = pos.unsqueeze(0)
pos

In [ ]:
pos.expand_as(x)

In [4]:
import torch
pos = torch.tensor([[0, 1, 2, 3, 4]])  # Shape: (1, 5)
x = torch.tensor([[1, 2, 3, 0, 0], [4, 5, 6, 7, 0]])  # Shape: (2, 5)
print(pos.shape)
pos = pos.unsqueeze(0)
print(pos.shape)

torch.Size([1, 5])
torch.Size([1, 1, 5])


In [5]:
pos = pos.expand_as(x)
print(pos.shape)

RuntimeError: expand(torch.LongTensor{[1, 1, 5]}, size=[2, 5]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (3)

In [22]:
import torch

def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0)  # batch_size x 1 x len_k(=len_q), one is masking
    print(pad_attn_mask)
    pad_attn_mask = pad_attn_mask.unsqueeze(1)
    print(pad_attn_mask)
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

# Example data
seq_q = torch.tensor([[1, 2, 3, 0, 0], [4, 5, 6, 7, 0]])  # Query sequences
seq_k = torch.tensor([[1, 2, 3, 4, 0], [5, 6, 7, 8, 9]])  # Key sequences

# Call the function

mask = get_attn_pad_mask(seq_q, seq_k)

# Print the result
print("Attention Pad Mask:")
print(mask)

print(mask.shape)
print(type(seq_q))
print(seq_q.size())
print(seq_q.data.eq(0))

tensor([[False, False, False, False,  True],
        [False, False, False, False, False]])
tensor([[[False, False, False, False,  True]],

        [[False, False, False, False, False]]])
Attention Pad Mask:
tensor([[[False, False, False, False,  True],
         [False, False, False, False,  True],
         [False, False, False, False,  True],
         [False, False, False, False,  True],
         [False, False, False, False,  True]],

        [[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]]])
torch.Size([2, 5, 5])
<class 'torch.Tensor'>
torch.Size([2, 5])
tensor([[False, False, False,  True,  True],
        [False, False, False, False,  True]])


In [27]:
seq_k = torch.tensor([[1, 2, 3, 4, 0], [5, 6, 7, 8, 9]])
pad_attn_mask = seq_k.data.eq(0)
print(pad_attn_mask.shape)
pad_attn_mask = pad_attn_mask.unsqueeze(1)
print(pad_attn_mask.shape)

torch.Size([2, 5])
torch.Size([2, 1, 5])


In [29]:
batch_size = 2

In [30]:
pad_attn_mask.expand(batch_size, len_q, len_k)

NameError: name 'len_q' is not defined

In [33]:
Q = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])

K = torch.tensor([[7, 8],
                  [9, 10],
                  [11, 12]])

V = torch.tensor([[0.1, 0.2],
                  [0.3, 0.4],
                  [0.5, 0.6]])

attn_mask = torch.tensor([[False, True],
                          [False, False]])


In [44]:
K.size(-2)

3

In [48]:
Q.transpose(-1,-2)

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [51]:
import torch

# Example attention scores tensor
scores = torch.tensor([[0.5, 0.8, 0.6],
                       [0.2, 0.4, 0.9]])

# Example attention mask tensor
attn_mask = torch.tensor([[False, False, True],
                          [False, True, True]])


# Mask out attention scores based on the mask
scores.masked_fill_(attn_mask, -1e9)


tensor([[ 5.0000e-01,  8.0000e-01, -1.0000e+09],
        [ 2.0000e-01, -1.0000e+09, -1.0000e+09]])

In [52]:
import torch

# Example attention scores tensor
scores = torch.tensor([[0.5, 0.8, -1.2],
                       [0.2, -0.4, 0.9]])

# Apply softmax along the last dimension
softmax_scores = torch.nn.Softmax(dim=-1)(scores)

print("Softmax Scores:")
print(softmax_scores)


Softmax Scores:
tensor([[0.3949, 0.5330, 0.0721],
        [0.2807, 0.1540, 0.5653]])


In [54]:
import torch

# Dummy input sequence with 5 tokens and 768-dimensional embeddings
input_sequence = torch.randn(5, 768)

# Projection matrices for query, key, and value
W_q = torch.randn(768, 64)  # Projection matrix for query
W_k = torch.randn(768, 64)  # Projection matrix for key
W_v = torch.randn(768, 64)  # Projection matrix for value

# Projecting input sequence into query, key, and value vectors
Q = input_sequence @ W_q  # Query vectors of shape (5, 64)
K = input_sequence @ W_k  # Key vectors of shape (5, 64)
V = input_sequence @ W_v  # Value vectors of shape (5, 64)

print("Shape of Query (Q) vectors:", Q.shape)
print("Shape of Key (K) vectors:", K.shape)
print("Shape of Value (V) vectors:", V.shape)


Shape of Query (Q) vectors: torch.Size([5, 64])
Shape of Key (K) vectors: torch.Size([5, 64])
Shape of Value (V) vectors: torch.Size([5, 64])


In [61]:
d_model = 768
d_ff = d_model * 4
d_k = d_v = 64

# Example code to create tensors with specified dimensions
import torch

# Position-wise FFN
pos_ffn_input = torch.randn(10, d_model)  # Example input tensor with shape [batch_size, d_model]
print(pos_ffn_input.shape)
linear_layer1 = torch.nn.Linear(d_model, d_ff)  # Linear projection to hidden layer size (d_ff)
linear_layer2 = torch.nn.Linear(d_ff, d_model)  # Linear projection back to d_model size

# Multi-head attention
n_heads = 8
seq_len = 10
input_sequence = torch.randn(10, seq_len, d_model)  # Example input tensor with shape [batch_size, seq_len, d_model]
linear_q = torch.nn.Linear(d_model, n_heads * d_k)  # Linear projection for queries
linear_k = torch.nn.Linear(d_model, n_heads * d_k)  # Linear projection for keys
linear_v = torch.nn.Linear(d_model, n_heads * d_v)  # Linear projection for values

print('q: ', linear_q)
print('k: ', linear_k)
print('v: ', linear_v)

torch.Size([10, 768])
q:  Linear(in_features=768, out_features=512, bias=True)
k:  Linear(in_features=768, out_features=512, bias=True)
v:  Linear(in_features=768, out_features=512, bias=True)
